In [1]:
import numpy as np
import healpy as hp

In [2]:
nside=256
freqs=np.arange(544,1088,step=1)
# nfreqs=len(freqs)
nfreqs=181
lmax=3*nside -1
almsize=hp.Alm.getsize(lmax)
r=3 #foreground model rank
nbins=76

In [3]:
noise_binned= np.load('noise_binned.npy')
hi_binned= np.load('hi_binned.npy')
c_hat=np.load('binned_empirical.npy')
print(hi_binned.shape)

(76, 181, 181)


In [4]:
def smica(params,hi, noise, empirical,  n_bins, nfreqs, r):
    
    # Reshape the flattened params back into F_init and P_b_init
    
    F = params[:nfreqs * r].reshape((nfreqs, r))
    P_b = params[nfreqs * r:].reshape((n_bins,r,r))
    
    
    """# === Normalize columns of F and rescale P accordingly ===
    for i in range(r):
        norm = np.linalg.norm(F[:, i])

        F[:, i] /= norm  # normalize column
        for b in range(n_bins):
            P_b[b, i, :] *= norm
            P_b[b, :, i] *= norm  # symmetric scaling"""
    
    
    # norm = np.linalg.norm(F, axis=0)
    
    # # Normalize F
    # F_normalized = F / norm        
    # # Adjust P_b: D^{-1} P_b D^{-1}
    # D_inv = np.diag(1.0 / norm)
    # for b in range(n_bins):
    #     P_b[b] = D_inv @ P_b[b] @ D_inv
                
    # === Compute the cost function ===            
    cost=0
    model=np.zeros((n_bins, nfreqs, nfreqs))
    model_inv=np.zeros((n_bins, nfreqs, nfreqs))
    
    # term=np.zeros(n_bins)
    for b in range(n_bins):
        model[b]= hi[b] + noise[b] + np.dot(F, np.dot(P_b[b], F.T))
        model_inv[b]= np.linalg.inv(model[b])
        
        sign, logdet= np.linalg.slogdet(np.dot(empirical[b],model_inv[b]))
        trace=np.trace(np.dot(empirical[b] , model_inv[b]))
        
        term = logdet + trace -nfreqs
        cost += term
        
    return cost

In [5]:
def jacobian_2(params,hi, noise,  empirical, n_bins, nfreqs, r):
    
    delta_ell=10
    # Reshape the flattened params back into F_init and P_b_init
    F = params[:nfreqs * r].reshape((nfreqs, r))
    P_b = params[nfreqs * r:].reshape((n_bins,r,r))
    
    """# === Normalize columns of F and rescale P accordingly ===
    for i in range(r):
        norm = np.linalg.norm(F[:, i])
        
        F[:, i] /= norm  # normalize column
        for b in range(n_bins):
            # Scale row i and column i of P_b[b] accordingly
            P_b[b, i, :] *= norm
            P_b[b, :, i] *= norm """
            
    # Compute norms of each column of F
    # norm = np.linalg.norm(F, axis=0)
    
    # # Normalize F
    # F_normalized = F / norm        
    # # Adjust P_b: D^{-1} P_b D^{-1}
    # D_inv = np.diag(1.0 / norm)
    # for b in range(n_bins):
    #     P_b[b] = D_inv @ P_b[b] @ D_inv
    
    grad_P=np.zeros_like(P_b) # Derivative wrt P_b
    grad_F=np.zeros_like(F) # Derivative wrt F
    
    for b in range(n_bins):
        
        R= np.dot(F, np.dot(P_b[b], F.T)) # model covariance depending on only unknown parameters
        """try:
            R_inv = np.linalg.inv(R)
        except np.linalg.LinAlgError:
            # Optional: add small regularization to make model invertible
            R += 1e-6 * np.identity(nfreqs)
            R_inv = np.linalg.inv(R)"""
        R_inv= np.linalg.inv(R)
        
        Delta= R_inv - np.dot(R_inv,np.dot(empirical[b], R_inv)) #nf x nf

        # ∂φ/∂P_q 
        grad_P[b,:,:]= np.dot(F.T, np.dot(Delta, F)) # (r,r)
        # ∂φ/∂F
        grad_F += 2*np.dot(Delta, np.dot(F, P_b[b])) # (nf,r)
        
    # Flatten the gradients
    grad_P_flat = grad_P.flatten()
    grad_F_flat = grad_F.flatten()
    print(grad_F_flat.shape)
    print(grad_P_flat.shape)
    grad_total= np.concatenate([grad_F_flat.flatten(), grad_P_flat.flatten()])
    return grad_total

In [5]:
def pca(empirical, r, n_bins):
    """
    Perform PCA on the covariance matrix.
    
    Parameters:
    ----------
    c_hat : np.ndarray
        Covariance matrix of shape (n_bins, n_freq, n_freq).
    
    r : int
        Number of principal components to retain.
    
    Returns:
    -------
    F : np.ndarray
        Matrix of shape (n_freq, r) containing the principal components.
    
    P_b : np.ndarray
        Matrix of shape (n_bins, r, r) containing the eigenvalues and eigenvectors.
    """
    # Perform PCA
    R_global = np.mean(empirical, axis=0)  # c_hat has shape (n_bins, n_freqs, n_freqs)
    #eigen decompositiom
    eigvals, eigvecs = np.linalg.eigh(R_global) #ascending order
    #Take the top r eigenvectors:
    F = eigvecs[:, -r:]  # largest eigenvectors

    P_b = np.zeros((n_bins, r, r))
    for b in range(n_bins):
        P_b[b] =np.dot( F.T , np.dot(empirical[b] , F))
        P_b[b] = 0.5 * (P_b[b] + P_b[b].T)
    return eigvals,F, P_b

In [6]:
e,F_pca, P_b_pca = pca(c_hat,r, nbins)
params_pca = np.concatenate([F_pca.flatten(), P_b_pca.flatten()])

In [7]:
print(F_pca.shape) 
print(P_b_pca.shape)
print(params_pca.shape)

(181, 3)
(76, 3, 3)
(1227,)


In [8]:
phi= smica(params_pca, hi_binned, noise_binned, c_hat, nbins, nfreqs, r)
print(phi)

-1707.294929961676


In [10]:
Grad= jacobian_2(params_pca, hi_binned, noise_binned, c_hat, nbins, nfreqs, r)
print(Grad)
print(Grad.shape)

(543,)
(684,)
[-1.62485676e+21 -3.88069569e+21 -8.56872485e+21 ... -7.64948241e+21
  4.05848519e+21  1.03084668e+22]
(1227,)


In [9]:
f = lambda x: smica(x,hi=hi_binned, noise=noise_binned, empirical=c_hat, n_bins=nbins, nfreqs=nfreqs, r=r)
grad_f = lambda x: jacobian_2(x,hi=hi_binned, noise=noise_binned, empirical=c_hat, n_bins=nbins, nfreqs=nfreqs, r=r)

In [10]:
def taylor_test(cost, grad_cost, params, epsilons=None):
    if epsilons is None:
        epsilons = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5] #stepsize

    np.random.seed(0)
    h = np.random.randn(*params.shape)
    h /= np.linalg.norm(h)  #unit vector

    f0 = cost(params) #at a point x0
    grad0 = grad_cost(params) 
    directional_derivative = np.dot(grad0.T, h)

    for eps in epsilons:
        f_eps = cost(params + eps * h)
        residual = abs(f_eps - f0 - eps * directional_derivative)
        print(f"epsilon = {eps:.1e}, Residual = {residual:.2e}, Residual/epsilon = {residual/eps:.2e}")


In [11]:
taylor_test(f, grad_f, params_pca) #3mins

NameError: name 'jacobian_2' is not defined

In [ ]:
from scipy.optimize import approx_fprime
approx_grad = approx_fprime(params_pca, f, epsilon=1e-6) #finite differences
print(approx_grad) #4mins

In [15]:
grad_f1 = lambda x: approx_fprime(x, f, epsilon=1e-6)

In [16]:
taylor_test(f, grad_f1, params_pca)

KeyboardInterrupt: 

In [ ]:
np.linalg.norm(approx_grad - grad_f(params_pca))

(543,)
(684,)


np.float64(9.051665227619901e+25)

In [ ]:
sync_ff=np.load('smoothed_maps_1_nohi.npy')